In [44]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


def results(results):
  print('Optimal Hyperparams: {}\n'.format(results.best_params_))
  means = results.cv_results_['mean_test_score']
  stds = results.cv_results_['std_test_score']

  for mean, std, params in zip(means, stds, results.cv_results_['params']):
    print('Mean {} Standard Deviation {} Hyperparameters {}'.format(round(mean,3), round(std * 2, 3), params))
   

def del_quantil(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    df = df[~((df[column] < (Q1 - 1.5 * IQR)) | (df[column] > (Q3 + 1.5 * IQR)))]

    return df    

In [45]:
df = pd.read_csv("../Data_Preprocessing/df_nnz_Km_clean.csv")

Разделяю данные на обучающую, тестовую и валидационную выборки. 

In [46]:
# Train Test Split
features = df.drop('Km', axis=1)
labels = df['Km']

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

Вывожу на экран доли данных в каждой выборке относительно исходных данных.

In [47]:
for data in [y_train, y_val, y_test]:
  print(round(len(data) / len(labels), 2))

0.6
0.2
0.2


Сохраняю данные в формате CSV. Он сохраняет обучающие, валидационные и тестовые данные, а также соответствующие им метки классов.

In [48]:
# Save the data
X_train.to_csv('../Data_ML/Km/train_features.csv', index=False)
X_val.to_csv('../Data_ML/Km/val_features.csv', index=False)
X_test.to_csv('../Data_ML/Km/test_features.csv', index=False)

y_train.to_csv('../Data_ML/Km/train_labels.csv', index=False)
y_val.to_csv('../Data_ML/Km/val_labels.csv', index=False)
y_test.to_csv('../Data_ML/Km/test_labels.csv', index=False)

Кросс-валидация

In [49]:
train_features = pd.read_csv('../Data_ML/Km/train_features.csv')
train_labels = pd.read_csv('../Data_ML/Km/train_labels.csv')

val_features = pd.read_csv('../Data_ML/Km/val_features.csv')
val_labels = pd.read_csv('../Data_ML/Km/val_labels.csv')

test_features = pd.read_csv('../Data_ML/Km/test_features.csv')
test_labels = pd.read_csv('../Data_ML/Km/test_labels.csv')

In [50]:
rf = RandomForestRegressor()

scores = cross_val_score(rf, train_features, train_labels.values.ravel(), cv=5)

In [51]:
scores

array([-5.07636035,  0.18583896,  0.45756295,  0.31711836,  0.14039398])

Настройка гиперпараметров

Выполняю кросс-валидацию с перебором сетки для регрессорной модели случайного леса

In [52]:
rf = RandomForestRegressor()

hyperparams = {
    'n_estimators': [5, 25, 50, 100],
    'max_depth': [2, 12, 24, None]
}

cross_val = GridSearchCV(rf, hyperparams, cv=5)
cross_val.fit(train_features, train_labels.values.ravel())

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 12, 24, None],
                         'n_estimators': [5, 25, 50, 100]})

In [53]:
GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 12, 24, None],
                         'n_estimators': [5, 25, 50, 100]})

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 12, 24, None],
                         'n_estimators': [5, 25, 50, 100]})

In [54]:
results(cross_val)

Optimal Hyperparams: {'max_depth': 2, 'n_estimators': 100}

Mean -2.493 Standard Deviation 9.831 Hyperparameters {'max_depth': 2, 'n_estimators': 5}
Mean -0.441 Standard Deviation 2.419 Hyperparameters {'max_depth': 2, 'n_estimators': 25}
Mean -0.398 Standard Deviation 2.588 Hyperparameters {'max_depth': 2, 'n_estimators': 50}
Mean -0.348 Standard Deviation 2.199 Hyperparameters {'max_depth': 2, 'n_estimators': 100}
Mean -1.2 Standard Deviation 4.139 Hyperparameters {'max_depth': 12, 'n_estimators': 5}
Mean -1.484 Standard Deviation 7.038 Hyperparameters {'max_depth': 12, 'n_estimators': 25}
Mean -0.933 Standard Deviation 4.949 Hyperparameters {'max_depth': 12, 'n_estimators': 50}
Mean -0.672 Standard Deviation 4.051 Hyperparameters {'max_depth': 12, 'n_estimators': 100}
Mean -1.246 Standard Deviation 6.408 Hyperparameters {'max_depth': 24, 'n_estimators': 5}
Mean -1.129 Standard Deviation 5.167 Hyperparameters {'max_depth': 24, 'n_estimators': 25}
Mean -1.373 Standard Deviation 6.512 

In [55]:
rf1 = RandomForestRegressor(n_estimators=5, max_depth=None)
rf1.fit(train_features, train_labels.values.ravel())

rf2 = RandomForestRegressor(n_estimators=25, max_depth=2)
rf2.fit(train_features, train_labels.values.ravel())

rf3 = RandomForestRegressor(n_estimators=50, max_depth=2)
rf3.fit(train_features, train_labels.values.ravel())

RandomForestRegressor(max_depth=2, n_estimators=50)

Эволюция модели

In [56]:
for mdl in [rf1, rf2, rf3]:
    y_pred = mdl.predict(val_features)
    mse = mean_squared_error(val_labels, y_pred)
    r2 = r2_score(val_labels, y_pred)
    mae = mean_absolute_error(val_labels, y_pred)
    rmse = np.sqrt(mse) 
    
    print('Max Depth: {} || Estimators: {} || MSE: {:.4f} || R-squared: {:.4f} || MAE: {:.4f} || RMSE: {:.4f}'.format(
        mdl.max_depth, mdl.n_estimators, mse, r2, mae, rmse))

Max Depth: None || Estimators: 5 || MSE: 85051.2301 || R-squared: -0.1460 || MAE: 50.7586 || RMSE: 291.6354
Max Depth: 2 || Estimators: 25 || MSE: 74029.1117 || R-squared: 0.0025 || MAE: 53.5043 || RMSE: 272.0829
Max Depth: 2 || Estimators: 50 || MSE: 75502.6924 || R-squared: -0.0173 || MAE: 54.6396 || RMSE: 274.7775


In [57]:
y_pred = mdl.predict(val_features)
mse = mean_squared_error(test_labels, y_pred)
r2 = r2_score(test_labels, y_pred)
mae = mean_absolute_error(test_labels, y_pred)
rmse = np.sqrt(mse)  # RMSE is the square root of MSE
    
print('Max Depth: {} || Estimators: {} || MSE: {:.4f} || R-squared: {:.4f} || MAE: {:.4f} || RMSE: {:.4f}'.format(
        rf1.max_depth, rf1.n_estimators, mse, r2, mae, rmse))

Max Depth: None || Estimators: 5 || MSE: 33984.9601 || R-squared: -0.2136 || MAE: 81.1415 || RMSE: 184.3501


In [58]:
df.describe()

,Km,X,IR,pot2,ph,temp,dstr,cryst,lgCmin,lgCmax,...,Kappa2_log,EState_VSA6_log,EState_VSA4_log,SMR_VSA7_log,Complexity1_log,TPSA_log,TPSA1_log,TPSA2_log,MaxEStateIndex.1_log,MaxEStateIndex.2_log
count,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,...,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000
mean,42.729274,2.534484,1.207227,0.281523,4.526969,33.173750,2.703571,5.983393,-1.289112,0.524355,...,1.035303,0.558000,0.930547,0.926984,2.988612,2.326087,3.932550,3.833522,1.794193,1.502810
std,191.919177,0.269944,0.178046,0.571944,1.203239,9.404625,0.656027,1.664337,1.403177,1.211070,...,1.299299,1.045894,1.228008,1.366343,2.683901,2.262861,0.486101,0.214840,0.254166,0.678240
min,0.000100,1.830000,0.700000,-1.358000,2.000000,15.000000,1.000000,0.000000,-4.000000,-4.000000,...,-1.579078,0.000000,0.000000,0.000000,0.000000,0.000000,3.701302,3.701302,0.000000,0.000000
25%,0.112000,2.280000,1.084000,-0.084100,4.000000,25.000000,3.000000,5.800000,-2.301030,-0.221849,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.701302,3.701302,1.790549,1.790549
50%,0.450000,2.614400,1.217200,0.067650,4.000000,31.000000,3.000000,7.000000,-1.301030,0.195609,...,0.686741,0.000000,0.000000,0.000000,5.420535,3.011113,3.951244,3.701302,1.790549,1.790549
75%,6.432250,2.750000,1.320000,0.662200,4.500000,40.000000,3.000000,7.000000,-0.301030,1.380211,...,1.934395,0.000000,2.552693,2.538049,5.420535,4.462915,3.951244,3.951244,1.791759,1.791759
max,3820.000000,3.170000,1.710000,1.690000,10.000000,90.000000,3.000000,7.000000,2.944483,3.354108,...,5.552913,4.756298,4.433535,5.175934,6.190315,7.051345,6.783325,6.783325,2.496743,2.521040


Определяю верхний и нижний порог для выбросов, с использованием межквартильного размаха. Считываю выбросы для каждого столбца.

In [59]:

Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = ((df < lower_bound) | (df > upper_bound)).sum()
outliers = outliers.sort_values(ascending=False)

In [60]:
outliers

XLogP                   257
EState_VSA6_log         255
Sufrace                 238
VSA_EState4             227
ph                      227
MaxEStateIndex.2_log    215
Km                      214
dstr                    208
MinAbsEStateIndex       174
cryst                   128
MaxEStateIndex.1_log     95
VSA_EState8              48
MinPartialCharge.1       46
lgCconst                 45
TPSA1_log                44
lgCcat                   43
IR                       35
MinEStateIndex           31
TPSA2_log                13
PEOE_VSA9_log            12
lgvolume                 10
MinPartialCharge          8
pot2                      7
lgCmax                    7
Kappa2_log                6
temp                      4
lgCmin                    1
PEOE_VSA8_log             0
EState_VSA4_log           0
SMR_VSA7_log              0
Complexity1_log           0
TPSA_log                  0
polym                     0
X                         0
BalabanJ                  0
BCUT2D_CHGLO        

In [61]:
df.shape

(1120, 36)

In [62]:
df = del_quantil(df, 'XLogP')

In [63]:
outliers

XLogP                   257
EState_VSA6_log         255
Sufrace                 238
VSA_EState4             227
ph                      227
MaxEStateIndex.2_log    215
Km                      214
dstr                    208
MinAbsEStateIndex       174
cryst                   128
MaxEStateIndex.1_log     95
VSA_EState8              48
MinPartialCharge.1       46
lgCconst                 45
TPSA1_log                44
lgCcat                   43
IR                       35
MinEStateIndex           31
TPSA2_log                13
PEOE_VSA9_log            12
lgvolume                 10
MinPartialCharge          8
pot2                      7
lgCmax                    7
Kappa2_log                6
temp                      4
lgCmin                    1
PEOE_VSA8_log             0
EState_VSA4_log           0
SMR_VSA7_log              0
Complexity1_log           0
TPSA_log                  0
polym                     0
X                         0
BalabanJ                  0
BCUT2D_CHGLO        